# UX Research Dashboard (Dash Plotly PoC 2021)
New PoC UX Research dashboard built using Dash Plotly, https://dash.plotly.com/
Created by Leslie A. McFarlin, Principal UX Architect @ Wheels

In [1]:
## Install jupyter-dash to work in a jupyter notebook
# https://dash.plotly.com/installation
# Comment out after installation completes to hide installation details
# !pip install jupyter-dash

In [2]:
## Begin necessary library imports
# Import file and data handling
import os
import pandas as pd
import numpy as np

# Import regex
import re

# Import relevant dash libraries
from jupyter_dash import JupyterDash
from dash import dcc
from dash import dash_table
from dash import html
from dash.dependencies import Input, Output

# Import plotly visualization support
import plotly.express as px

## Data Handling
This step refers to importing the data and cleaning it. The data used is downloaded from the UX Team's UX ResearchOps pages on Sharepoint. It begins by importing a CSV into pandas, creating a dataframe, and then removing any duplicates and tracking missing data.

In [3]:
## Find files
# Filepath
filepath = "/Users/lesli/OneDrive/Desktop/Python Projects/Plotly Dashboard/data"

# Get list of files
files = [file for file in os.listdir(filepath)]

In [4]:
## Create dataframe from CSV
participants = pd.read_csv(os.path.join(filepath, files[3]))

### Participants Dataframe
This dataframe has the richest amount of data to draw from, and so this PoC will start here.

In [5]:
# Check participants dataframe
participants.head(5)

,Project Code,Type,Year,Month,Product/Feature/Topic
0,SRC-002,Internal,2020,August,Search
1,SRC-002,Internal,2020,August,Search
2,SRC-002,External,2020,August,Search
3,SRC-002,External,2020,August,Search
4,SRC-002,External,2020,August,Search


In [6]:
## Begin cleaning the data
# Gather column names into a list and convert to lowercase
columns = [c for c in participants.columns]

# Maps column names
column_map = {}

# Iterate through column list
for col in columns:
    # Set the dictionary key to the actual column name
    key = col
    # Get the column name from the actual column name to process as the dictionary value
    col_name = col
    # If there is a space in the column name
    if re.search(r"\s", col_name):
        # Replace space with underscore
        col_name = col_name.replace(" ", "_")
    # If there is a non-alphanumeric in the column name
    elif re.search(r"\W+", col_name):
        # Extract the first part of the string
        col_name = re.findall("[\dA-Za-z]*", col)[0]
    # If no space or alphanumeric
    else:
        # Nothing needed
        col_name = col
    # Add to mapping dictionary
    column_map[key] = col_name.lower()

# View new column names
print(column_map)
    
# Rename all columns
participants.rename(columns = column_map, inplace = True)

# Check dataframe
participants.head(5)

{'Project Code': 'project_code', 'Type': 'type', 'Year': 'year', 'Month': 'month', 'Product/Feature/Topic': 'product'}


,project_code,type,year,month,product
0,SRC-002,Internal,2020,August,Search
1,SRC-002,Internal,2020,August,Search
2,SRC-002,External,2020,August,Search
3,SRC-002,External,2020,August,Search
4,SRC-002,External,2020,August,Search


## App Initialization
This dashboard PoC will run locally in this JupyterLab Notebook. As such, styling will be done locally for the purposes of this PoC. Also, the dashboard items will appear in an iFrame in this notebook. A future PoC will explore running an external, fully styled version of the same data.

In [14]:
## Initialize dashboard applications

# Create the app itself
app = JupyterDash(__name__)

In [15]:
## Set up the test app layout
app.layout = html.Div([html.H1("JupyterDash PoC", 
                               style = {"color": "#696868", 
                                        "font-family": "'proxima-nova', sans-serif", 
                                        "font-weight": 700}), # Main Title
                       ## FIRST GRAPH SECTION
                       html.Label(["Year", 
                                  dcc.Dropdown(id = "dropdown_year", 
                                               clearable = False, 
                                               value = 2021, 
                                               options = [{"label": "2021", "value": 2021},
                                                          {"label": "2020", "value": 2020}],# Values must match datatype in dataframe
                                               style = {"color": "#696868", 
                                                        "font-family":"'proxima-nova', sans-serif", 
                                                        "font-size": "13px", "border": "1px solid #929393", 
                                                        "border-radius": "4px"} ) 
                                  ]), # Defines label and dropdown to filter view of histogram
                       dcc.Graph(id = "histogram_participant_type"), # Only define figure property here if figure content will not change
                       ## SECOND GRAPH SECTION
                       html.H2("Static Graph Example", 
                               style = {"color": "#696868", 
                                        "font-family": "'proxima-nova', sans-serif", 
                                        "font-weight": 700} ), # Section Title
                       dcc.Graph(id = "histogram_product", 
                                 figure = px.histogram(participants, 
                                                       x = "product", 
                                                       title = "Participant Counts by Research Topic",
                                                       color = "product",
                                                       color_discrete_sequence=['#56A0D3','#679146', '#EC881D', '#ABB518', '#7C2B83', '#FEC057', '#85CDDB'],
                                                       template = "simple_white"), 
                                 style = {"color": "#696868", 
                                          "font-family": "'proxima-nova', sans-serif", 
                                          "font-weight": 300},
                                 ), 
                       ## DYNAMIC DATA TABLE SECTION
                       html.H2("Dash DataTable Example",
                               style = {"color": "#696868", 
                                        "font-family": "'proxima-nova', sans-serif", 
                                        "font-weight": 700}), # Section Title
                       dash_table.DataTable(id = "datatable_participants_dataframe",
                                            data = participants.to_dict('records'),
                                            columns = [{"name": "Project Code", "id": "project_code", "type": "text", "editable": False},
                                                       {"name": "Year", "id": "year", "type": "numeric", "editable": False},
                                                       {"name": "Month", "id": "month", "type": "text", "editable": False},
                                                       {"name": "Research Topic", "id": "product", "type": "any", "editable": False}],
                                            sort_action = "native",
                                            filter_action = "native",
                                            style_cell = {"color": "#696868", 
                                                          "font-family": "'proxima-nova', sans-serif", 
                                                          "font-weight": 300,
                                                          "textAlign": "left",
                                                          "paddingLeft": "18px"},
                                            style_header = {"backgroundColor": "#ffffff",
                                                            "fontWeight": "bold",
                                                            "borderTop": "4px solid #d2d2d2",
                                                            "borderBottom": "4px solid #679146"},
                                            style_data_conditional = [{"if": {"row_index": "odd"},
                                                                        "backgroundColor": "#f0f0f0"}],
                                            style_as_list_view = True
                                            )
                        ])

## Define callback to update graph
# Callback decorator
# output is the figure property of the graph
# input is from the relevant dropdown's value property
@app.callback(Output(component_id = "histogram_participant_type", component_property = "figure"), 
              [Input(component_id = "dropdown_year", component_property = "value")])

# Callback function to update the graph based on value of Year dropdown
def updateFigure(dropdown_year):
    ## Return the histogram with categorical data from type column
    # Begin by filtering the dataframe so that only the relevant rows appear
    # The filter value to match is the argument passed to this function
    participants_year_copy = participants[participants["year"] == dropdown_year]
    # Create the updated figure based on the filter above
    this_fig = px.histogram(participants_year_copy, x = "type", category_orders = dict(type = ["External", "Internal"]), title = "Participant Type", color = "type", color_discrete_sequence=['#56A0D3','#679146', '#EC881D', '#ABB518', '#7C2B83', '#FEC057', '#85CDDB'], template = "simple_white")
    # Return the updated figure
    return this_fig

## Run the app
# mode = "inline" will run within the notebook
# mode = "external" will generate a local URL
# mode = "jupyterlab" will generate a tab within Jupyter
app.run_server(mode="inline", port = 8040)

In [13]:
# run_server() with a specified port is because of a known issue where re-running on the same port produces an OSError that says the localhost is already in use.
# Uncomment the line below and run this cell if needing to push updates to the histogram above.
del app